In [25]:
from IPython.display import display, HTML 
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

<b><font size="5" color="red">ch09. 파일 io(입출력) 프로그래밍 </font><b>
파일 : txt, pickle, csv, json, hdf5(h5)

# 1절. txt파일에 데이터 저장하고 불러오기

In [26]:
try:
    f = open('data/ch09.txt', 'w')
except FileNotFoundError as e:
    print(e)
finally:
    f.close()

In [27]:
f = open('data/ch09.txt', 'w')
print('쓰기 가능한지 여부 :', f.writable())

쓰기 가능한지 여부 : True


In [28]:
f.write('Hello\nWorld')
f.close() # close() 쓰기한 내용이 저장에 필수!!!

In [30]:
# mode : 'r' 또는 'rt' : 파일이 없는 경우 예외, 파일이 있는 경우 text읽기 모드
#        'rb' : 바이너리 읽기 모두
#        'w' 또는 'wt' : text 쓰기 모드 (파일이 있으면 덮어씀, 파일이 없으면 파일 생성)
#        'wb' : 바이너리 쓰기 모드
#        'a' 또는 'at' : text 추가 모드 (파일이 있으면 append, 파일이 없으면 파일 생성)
#        읽기모드에서는 파일이 없으면 예외 / 쓰기모드에서는 폴더가 없으면 예외
# encoding
#    eur-kr (한글완성형) '믜' 표현 안됨
#    cp949 (확장된 한글완성형) : open()함수 기본 encoding방식(win)
#    utf-8 (한글조합형) : open()함수 기본 encoding방식(mac, linux), 주피터노트북 기본
with open('data/ch09.txt', 'w') as f:
    print('쓰기 가능한지 여부 :', f.writable())
    f.write('Hello\nPython\n')

쓰기 가능한지 여부 : True


In [31]:
with open('data/ch09.txt', 'a', encoding='cp949') as f:
    # 1방법
    f.write('홍길동, 33, 아무동9\n')
    f.write('김길동, 33, 아무동9\n')
    # 2방법
    textlist = ['홍길동, 33, 아무동9\n', '김길동, 33, 아무동9\n']
    for line in textlist:
        f.write(line)
    # 3방법
    f.writelines(textlist)

In [32]:
# 한줄씩 일기
with open('data/ch09.txt', 'r') as f:
    line = f.readline()
    print(line)

Hello



In [33]:
# 모든 줄을 일기
with open('data/ch09.txt', 'r') as f:
    line = f.readline()
    while line != '':
        print(line, end='')
        line = f.readline()

Hello
Python
홍길동, 33, 아무동9
김길동, 33, 아무동9
홍길동, 33, 아무동9
김길동, 33, 아무동9
홍길동, 33, 아무동9
김길동, 33, 아무동9


In [34]:
# 모두 일기
with open('data/ch09.txt', 'r') as f:
    lines = f.read()
    print(lines)

Hello
Python
홍길동, 33, 아무동9
김길동, 33, 아무동9
홍길동, 33, 아무동9
김길동, 33, 아무동9
홍길동, 33, 아무동9
김길동, 33, 아무동9



In [35]:
lines

'Hello\nPython\n홍길동, 33, 아무동9\n김길동, 33, 아무동9\n홍길동, 33, 아무동9\n김길동, 33, 아무동9\n홍길동, 33, 아무동9\n김길동, 33, 아무동9\n'

# 2절.  피클을 이용한 객체 저장 및 불러오기
## 2.1 형식이 있는 txt데이터 불러오기

In [36]:
class Member:
    def __init__(self, name, age, email, address):
        self.name = name
        self.age = age
        self. email = email
        self. address = address
    def __str__(self):
        return '{}, {}, {}, {}'.format(self.name,
                                      '성년' if self.age>18 else '미성년',
                                      self.email,
                                      self.address)
    def as_dict(self):
        return {
            'name':self.name,
            'age':self.age,
            'email':self.email,
            'address':self.address
            }
    def __eq__(self, other):
        return self.name == other.name and \
                self.age == other.age and \
                self.email == other.email and \
                self.address == other.address

In [37]:
user1 = Member('홍', 20, 'a@a.com', '신림동')
user2 = Member('홍', 20, 'a@a.com', '신림동')
print(user1==user2)
print(user1.__eq__(user2)) # 같은 내용이면 True
id(user1), id(user2), type(user1), type(user2)

True
True


(2126539117792, 2126539119712, __main__.Member, __main__.Member)

In [41]:
# 형식이 있는 txt 파일 내용을 member list(피클 저장), 딕션너리 list(데이터프레임)로 저장
user_list = [] # member list
user_dict = [] # 딕셔너리 list

with open('data/ch09_member.txt', 'r', encoding='utf-8') as txt_file:
    lines = txt_file.readlines()
# print(lines)
for line in lines:
#    print(line)
    data = line.split(',')
#    print(data)
    name = data[0]
    age = int(data[1].strip()) # strip() : 좌우 공백(space, \t, \n) 제거
    email = data[2].strip()
    address = data[3].strip()
    
    user = Member(name, age, email, address)
    user_list.append(user)
    user_dict.append(user.as_dict())
#    user_dict.append(user.__dict__) # 위와 동일 기능


In [42]:
for user in user_list:
    print(user)
user_dict

홍길동, 성년, kildong@hong.com, 서울시 관악구
홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
신길동, 성년, shinkil@hong.com, 서울시 동작구


[{'name': '홍길동', 'age': 20, 'email': 'kildong@hong.com', 'address': '서울시 관악구'},
 {'name': '홍길숙',
  'age': 25,
  'email': 'kilsuk1@hong.com',
  'address': '서울시 영등포구'},
 {'name': '신길동', 'age': 30, 'email': 'shinkil@hong.com', 'address': '서울시 동작구'}]

## 2.2 피클링
- 객체 리스트(user_list) -> 피클파일로 쓰기
- 피클 파일을 읽기 -> 객체 리스트(load_user_list)

In [43]:
# 피클링을 이용한 객체를 저장하기
import pickle
with open('data/ch09_member.data', 'wb') as f:
    pickle.dump(user_list, f)

In [44]:
# 피클링을 이용한 파일에서 객체 데이터로 읽기
with open('data/ch09_member.data', 'rb') as f:
    load_user_list = pickle.load(f)

In [45]:
user_list == load_user_list

True

In [46]:
for idx in range(len(user_list)):
    print(idx, user_list[idx])
    print(idx, load_user_list[idx])

0 홍길동, 성년, kildong@hong.com, 서울시 관악구
0 홍길동, 성년, kildong@hong.com, 서울시 관악구
1 홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
1 홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
2 신길동, 성년, shinkil@hong.com, 서울시 동작구
2 신길동, 성년, shinkil@hong.com, 서울시 동작구


In [47]:
for user, load_user in zip(user_list, load_user_list):
    print(user)
    print(load_user)

홍길동, 성년, kildong@hong.com, 서울시 관악구
홍길동, 성년, kildong@hong.com, 서울시 관악구
홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
신길동, 성년, shinkil@hong.com, 서울시 동작구
신길동, 성년, shinkil@hong.com, 서울시 동작구


In [48]:
for idx, (user, load_user) in enumerate(zip(user_list, load_user_list)):
    print(idx, user)
    print(idx, load_user)
    print(user==load_user)

0 홍길동, 성년, kildong@hong.com, 서울시 관악구
0 홍길동, 성년, kildong@hong.com, 서울시 관악구
True
1 홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
1 홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
True
2 신길동, 성년, shinkil@hong.com, 서울시 동작구
2 신길동, 성년, shinkil@hong.com, 서울시 동작구
True


In [49]:
result = []
for user, load_user in zip(user_list, load_user_list):
    result.append(user==load_user)
all(result)

True

# 3절. CSV형식 파일 읽기/쓰기
- csv파일 <--> 리스트데이터   3.1 / 3.2
- csv파일 <--> 딕셔너리데이터 3.3 / 3.4

## 3.1 reader

In [69]:
import csv
with open('data/ch09_member1.csv', 'r', encoding='utf-8')as f:
    reader = csv.reader(f)
    result = list(reader)
print(result)

[['홍길동', '20', 'kildong@hong.com', '서울시 관악구'], ['김길동', '40', 'kimdong@hong.com', '서울시 영등포구'], ['신길동', '30', 'sindong@hong.com', '서울시 동작구']]


In [70]:
# ""(따옴표)가 없는 데이터는 numeric으로
import csv
with open('data/ch09_member1.csv', 'r', encoding='utf-8')as f:
    reader = csv.reader(f,
                       quoting=csv.QUOTE_NONNUMERIC)
    result = list(reader)
print(result)

[['홍길동', 20.0, 'kildong@hong.com', '서울시 관악구'], ['김길동', 40.0, 'kimdong@hong.com', '서울시 영등포구'], ['신길동', 30.0, 'sindong@hong.com', '서울시 동작구']]


In [72]:
import csv
with open('data/ch09_member1.csv', 'r', encoding='utf-8')as f:
    reader = csv.reader(f,
                       quoting=csv.QUOTE_NONNUMERIC)
    result = list(reader)
dict_list = []
for data in result:
    dict_list.append({
        'name':data[0],
        'age':int(data[1]),
        'email':data[2],
        'address':data[3]
    })
dict_list

[{'name': '홍길동', 'age': 20, 'email': 'kildong@hong.com', 'address': '서울시 관악구'},
 {'name': '김길동',
  'age': 40,
  'email': 'kimdong@hong.com',
  'address': '서울시 영등포구'},
 {'name': '신길동', 'age': 30, 'email': 'sindong@hong.com', 'address': '서울시 동작구'}]

## 3.2 writer

In [74]:
user_list = [['홍길동', 20.0, 'kildong@hong.com', '서울시 관악구'],
             ['김길동', 40.0, 'kimdong@hong.com', '서울시 영등포구'],
             ['신길동', 30.0, 'sindong@hong.com', '서울시 동작구']]

In [75]:
with open('data/ch09_member1_write.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(user_list)

In [76]:
import csv
with open('data/ch09_member1_write.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, quoting=csv.QUOTE_NONNUMERIC)
    writer.writerows(user_list)

## 3.3 DictReader

In [95]:
import csv
with open('data/ch09_member3.csv', 'r', encoding='utf-8') as f:
    dict_reader = csv.DictReader(f)
    dict_list = list(dict_reader)
    print(dict_list)

for row in dict_list:
    if row['job'] is None:
        print(row['name'], row['age'], row['email'], row['address'])
    else:
        print(row['name'], row['age'], row['email'], row['address'], row['job'])

[{'name': '홍길동', 'age': '20', 'email': 'h@h.com', 'address': '서울시 관악구', 'job': None}, {'name': '신길동', 'age': '40', 'email': 's@h.com', 'address': '서울시 영등포구', 'job': '팀장'}, {'name': '김길동', 'age': '30', 'email': 'k@h.com', 'address': '서울시 동작구', 'job': None}]
홍길동 20 h@h.com 서울시 관악구
신길동 40 s@h.com 서울시 영등포구 팀장
김길동 30 k@h.com 서울시 동작구


In [96]:
# header가 없는 csv파일(ch09_member1-cp.csv)을 딕셔너리로 읽기
with open('data/ch09_member1-cp.csv', encoding='cp949') as f:
    dict_reader = csv.DictReader(f, 
                                 fieldnames=['Name', 'Age', 'Email', 'Address', 'Job'], 
                                 restkey='Job')
    result = list(dict_reader)
    print(result)
    
for row in result:
    print(row)

[{'Name': '홍길동', 'Age': '20', 'Email': 'kildong@hong.com', 'Address': '서울시 관악구', 'Job': ''}, {'Name': '김길동', 'Age': '40', 'Email': 'kimdong@hong.com', 'Address': '서울시 영등포구', 'Job': '팀장'}, {'Name': '신길동', 'Age': '30', 'Email': 'sindong@hong.com', 'Address': '서울시 동작구', 'Job': ''}]
{'Name': '홍길동', 'Age': '20', 'Email': 'kildong@hong.com', 'Address': '서울시 관악구', 'Job': ''}
{'Name': '김길동', 'Age': '40', 'Email': 'kimdong@hong.com', 'Address': '서울시 영등포구', 'Job': '팀장'}
{'Name': '신길동', 'Age': '30', 'Email': 'sindong@hong.com', 'Address': '서울시 동작구', 'Job': ''}


## 3.4 DictWriter
- 딕셔너리 리스트 -> CSV파일

In [97]:
user1 = {'name': '홍길동', 'age': 22, 'email': 'a@a.com', 'address': '신림동'}
user2 = {'name': '신길동', 'age': 30, 'email': 'b@a.com', 'address': '신림동'}
user3 = {'name': '김길동', 'age': 42, 'email': 'c@a.com', 'address': '신림동'}
user_list = [user1, user2, user3]
fieldnames = list(user1.keys())

In [99]:
list(user1.keys())

['name', 'age', 'email', 'address']

In [101]:
with open('data/ch09_member4.csv', 'w', encoding='utf-8', newline='') as f:
    dict_writer = csv.DictWriter(f,
                                fieldnames=fieldnames)
    dict_writer.writeheader() # header 쓰기
    dict_writer.writerows(user_list)

In [102]:
with open('data/ch09_member4.csv', 'a', encoding='utf-8', newline='') as f:
    dict_writer = csv.DictWriter(f,
                                fieldnames=fieldnames)
    # dict_writer.writeheader() # header 쓰기
    dict_writer.writerows(user_list)

## csv <-> 데이터프레임

In [103]:
import pandas as pd
member = pd.read_csv('data/ch09_member3.csv') # encoding='utf-8' : default라 불필요
member

,name,age,email,address,job
0,홍길동,20,h@h.com,서울시 관악구,NaN
1,신길동,40,s@h.com,서울시 영등포구,팀장
2,김길동,30,k@h.com,서울시 동작구,NaN


In [104]:
type(member)

pandas.core.frame.DataFrame

# 4절. JSON 데이터 저장하고 불러오기(dump, load)
- 딕셔너리리스트 <-> JSON 파일(기본)
- 객체리스트 <-> JSON 파일
## 4.1 dump(파일 출력)

In [105]:
data = [{'Name': '홍길동', 'Age': '20', 'Email': 'kildong@hong.com', 'Address': '서울', 'Job': ''},
        {'Name': '김길동', 'Age': '40', 'Email': 'kimdong@hong.com', 'Address': '인천', 'Job': '팀장'},
        {'Name': '신길동', 'Age': '30', 'Email': 'sindong@hong.com', 'Address': '경기', 'Job': ''}]

In [107]:
# ensure_ascii 매개변수
    # True  : 비ASCII문은 유니코드 형태로 저장
    # False : 비 ASII문자 원래 형태로 저장
import json
with open('data/ch09_member.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(data, # 딕셔너리 리스트
              jsonfile,
              ensure_ascii=False,
              indent='\t')

In [109]:
class Member:
    def __init__(self, name, age, email, address):
        self.name = name
        self.age = age
        self. email = email
        self. address = address
    def __str__(self):
        return '{}, {}, {}, {}'.format(self.name,
                                      self.age,
                                      self.email,
                                      self.address)
    def as_dict(self):
        return {
            'name':self.name,
            'age':self.age,
            'email':self.email,
            'address':self.address
            }
    def __eq__(self, other):
        if isinstance(other, Member):
            return self.__str__() == other.__str__()
#             return self.name == other.name and \
#                     self.age == other.age and \
#                     self.email == other.email and \
#                     self.address == other.address
        else:
            return False

In [112]:
user1 = Member('홍길동', 22, 'a@a.com', '신림동')
user2 = Member('홍길동', 22, 'a@a.com', '신림동')
print(user1.__eq__(user2))
print(user1=='홍길동')

True
False


In [113]:
member_list = [Member('홍길동', 22, 'a@a.com', '서울'),
              Member('신길동', 32, 'a@a.com', '서울'),
              Member('김길동', 42, 'a@a.com', '서울')]

In [116]:
with open('data/ch09_member1.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(member_list, # 객체리스트
             jsonfile,
             ensure_ascii=False,
             indent='\t',
             default=Member.as_dict # 객체를 딕션너리로 return하는 인스턴스 함수
             )

## 4.2 load(파일입력)
- json파일 -> 딕셔너리리스트(기본)
                 ↓
- json파일 -> 객체리스트

In [123]:
def as_member(dic): 
    '매개변수로 딕셔너리를 받아 Member 객체를 return'
    return Member(dic.get('name'),dic['age'],dic.get('email'),dic.get('address'))

In [124]:
member = as_member({'name':'김길동', 'age':20, 'email':'z@a.com', 'address':'관악'})
print(member)

김길동, 20, z@a.com, 관악


In [126]:
with open('data/ch09_member.json', 'r', encoding='utf-8') as f:
    load_data = json.load(f)
load_data

[{'Name': '홍길동',
  'Age': '20',
  'Email': 'kildong@hong.com',
  'Address': '서울',
  'Job': ''},
 {'Name': '김길동',
  'Age': '40',
  'Email': 'kimdong@hong.com',
  'Address': '인천',
  'Job': '팀장'},
 {'Name': '신길동',
  'Age': '30',
  'Email': 'sindong@hong.com',
  'Address': '경기',
  'Job': ''}]

In [132]:
with open('data/ch09_member1.json', 'r', encoding='utf-8') as f:
    load_member_list = json.load(f, object_hook=as_member)
for load_member in load_member_list:
    print(load_member)

홍길동, 22, a@a.com, 서울
신길동, 32, a@a.com, 서울
김길동, 42, a@a.com, 서울


### JSON -> 데이터 프레임

In [128]:
import pandas as pd
pd.read_json('data/ch09_member1.json')
# pd.함수() : encoding='utf-8' 기본값
# open(파일, 모드)함수 : encoding='cp949' 기본값

,name,age,email,address
0,홍길동,22,a@a.com,서울
1,신길동,32,a@a.com,서울
2,김길동,42,a@a.com,서울


In [130]:
pd.read_json('data/ch09_member.json', encoding='utf-8')

,Name,Age,Email,Address,Job
0,홍길동,20,kildong@hong.com,서울,
1,김길동,40,kimdong@hong.com,인천,팀장
2,신길동,30,sindong@hong.com,경기,


# 5절. hdf5(h5) 파일 쓰기/읽기
- h5py 사용
## 5.1 hdf5 파일 쓰기
- 딕셔너리 리스트를 hdf5 파일에 쓰기
- 데이터프레임을 hdf5파일에 쓰기

In [1]:
# seaborn : 시각화 패키지. 학습을 위한 데이타 셋 다수
import seaborn as sns
iris_df = sns.load_dataset('iris')
display(iris_df[::5])

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica


In [2]:
iris_df.to_hdf('data/ch09_iris.hdf5', key='iris')

In [16]:
iris_dic = iris_df.to_dict()
type(iris_dic), type(iris_df)

(dict, pandas.core.frame.DataFrame)

In [17]:
iris_dic.keys()

dict_keys(['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'])

In [18]:
for group, value in iris_dic.items():
    print(('그룹명:'), group)
    for key, data in value.items():
        print('{}:{}'.format(str(key), data), end='\t')
    print() # 개행

그룹명: sepal_length
0:5.1	1:4.9	2:4.7	3:4.6	4:5.0	5:5.4	6:4.6	7:5.0	8:4.4	9:4.9	10:5.4	11:4.8	12:4.8	13:4.3	14:5.8	15:5.7	16:5.4	17:5.1	18:5.7	19:5.1	20:5.4	21:5.1	22:4.6	23:5.1	24:4.8	25:5.0	26:5.0	27:5.2	28:5.2	29:4.7	30:4.8	31:5.4	32:5.2	33:5.5	34:4.9	35:5.0	36:5.5	37:4.9	38:4.4	39:5.1	40:5.0	41:4.5	42:4.4	43:5.0	44:5.1	45:4.8	46:5.1	47:4.6	48:5.3	49:5.0	50:7.0	51:6.4	52:6.9	53:5.5	54:6.5	55:5.7	56:6.3	57:4.9	58:6.6	59:5.2	60:5.0	61:5.9	62:6.0	63:6.1	64:5.6	65:6.7	66:5.6	67:5.8	68:6.2	69:5.6	70:5.9	71:6.1	72:6.3	73:6.1	74:6.4	75:6.6	76:6.8	77:6.7	78:6.0	79:5.7	80:5.5	81:5.5	82:5.8	83:6.0	84:5.4	85:6.0	86:6.7	87:6.3	88:5.6	89:5.5	90:5.5	91:6.1	92:5.8	93:5.0	94:5.6	95:5.7	96:5.7	97:6.2	98:5.1	99:5.7	100:6.3	101:5.8	102:7.1	103:6.3	104:6.5	105:7.6	106:4.9	107:7.3	108:6.7	109:7.2	110:6.5	111:6.4	112:6.8	113:5.7	114:5.8	115:6.4	116:6.5	117:7.7	118:7.7	119:6.0	120:6.9	121:5.6	122:7.7	123:6.3	124:6.7	125:7.2	126:6.2	127:6.1	128:6.4	129:7.2	130:7.4	131:7.9	132:6.4	133:6.3	134:6.1	135:7.7	136:

In [19]:
# 딕셔너리 리스트를 hdf파일로 쓰기
import h5py
with h5py.File('data/ch09_iris_dic.h5', 'w') as f:
    for group, value in iris_dic.items():
        grp = f.create_group(group)
        for key, data in value.items():
            grp.create_dataset(str(key), data=data)

## 5.2 hdf5 파일 읽기
- hdf파일을 딕셔너리 리스트로 읽기
- hdf파일을 데이터프레임으로 읽기

In [5]:
import pandas as pd
load_iris_df = pd.read_hdf('data/ch09_iris.hdf5', key='iris')
load_iris_df[::50]

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica


In [7]:
all(iris_df == load_iris_df) # 두 데이터프레임 변수의 모든 행과 열의 값이 같은지 여부

True

In [21]:
with h5py.File('data/ch09_iris_dic.h5', 'r') as f:
    for group, value in f.items():
        print('그룹 : ', group)
        for key, data in value.items():
            print('{}:{}'.format(key, data[()]), end='\t')
            # data[()] : h5 Dataset타입을 가져올 때

그룹 :  petal_length
0:1.4	1:1.4	10:1.5	100:6.0	101:5.1	102:5.9	103:5.6	104:5.8	105:6.6	106:4.5	107:6.3	108:5.8	109:6.1	11:1.6	110:5.1	111:5.3	112:5.5	113:5.0	114:5.1	115:5.3	116:5.5	117:6.7	118:6.9	119:5.0	12:1.4	120:5.7	121:4.9	122:6.7	123:4.9	124:5.7	125:6.0	126:4.8	127:4.9	128:5.6	129:5.8	13:1.1	130:6.1	131:6.4	132:5.6	133:5.1	134:5.6	135:6.1	136:5.6	137:5.5	138:4.8	139:5.4	14:1.2	140:5.6	141:5.1	142:5.1	143:5.9	144:5.7	145:5.2	146:5.0	147:5.2	148:5.4	149:5.1	15:1.5	16:1.3	17:1.4	18:1.7	19:1.5	2:1.3	20:1.7	21:1.5	22:1.0	23:1.7	24:1.9	25:1.6	26:1.6	27:1.5	28:1.4	29:1.6	3:1.5	30:1.6	31:1.5	32:1.5	33:1.4	34:1.5	35:1.2	36:1.3	37:1.4	38:1.3	39:1.5	4:1.4	40:1.3	41:1.3	42:1.3	43:1.6	44:1.9	45:1.4	46:1.6	47:1.4	48:1.5	49:1.4	5:1.7	50:4.7	51:4.5	52:4.9	53:4.0	54:4.6	55:4.5	56:4.7	57:3.3	58:4.6	59:3.9	6:1.4	60:3.5	61:4.2	62:4.0	63:4.7	64:3.6	65:4.4	66:4.5	67:4.1	68:4.5	69:3.9	7:1.5	70:4.8	71:4.0	72:4.9	73:4.7	74:4.3	75:4.4	76:4.8	77:5.0	78:4.5	79:3.5	8:1.4	80:3.8	81:3.7	82:3.9	83:5.1	84:4.5	85

In [40]:
load_iris_dic = {}
with h5py.File('data/ch09_iris_dic.h5', 'r') as f:
    for group, value in f.items():
        temp = {}
        for key, data in value.items():
            temp[int(key)] = data[()]
        load_iris_dic[group] = temp
        

# load_iris_dic['sepal_length'] = {0:5.4, 1:4}
# load_iris_dic['sepal_width'] = {0:5.4, 1:4}
load_iris_dic

{'petal_length': {0: 1.4,
  1: 1.4,
  10: 1.5,
  100: 6.0,
  101: 5.1,
  102: 5.9,
  103: 5.6,
  104: 5.8,
  105: 6.6,
  106: 4.5,
  107: 6.3,
  108: 5.8,
  109: 6.1,
  11: 1.6,
  110: 5.1,
  111: 5.3,
  112: 5.5,
  113: 5.0,
  114: 5.1,
  115: 5.3,
  116: 5.5,
  117: 6.7,
  118: 6.9,
  119: 5.0,
  12: 1.4,
  120: 5.7,
  121: 4.9,
  122: 6.7,
  123: 4.9,
  124: 5.7,
  125: 6.0,
  126: 4.8,
  127: 4.9,
  128: 5.6,
  129: 5.8,
  13: 1.1,
  130: 6.1,
  131: 6.4,
  132: 5.6,
  133: 5.1,
  134: 5.6,
  135: 6.1,
  136: 5.6,
  137: 5.5,
  138: 4.8,
  139: 5.4,
  14: 1.2,
  140: 5.6,
  141: 5.1,
  142: 5.1,
  143: 5.9,
  144: 5.7,
  145: 5.2,
  146: 5.0,
  147: 5.2,
  148: 5.4,
  149: 5.1,
  15: 1.5,
  16: 1.3,
  17: 1.4,
  18: 1.7,
  19: 1.5,
  2: 1.3,
  20: 1.7,
  21: 1.5,
  22: 1.0,
  23: 1.7,
  24: 1.9,
  25: 1.6,
  26: 1.6,
  27: 1.5,
  28: 1.4,
  29: 1.6,
  3: 1.5,
  30: 1.6,
  31: 1.5,
  32: 1.5,
  33: 1.4,
  34: 1.5,
  35: 1.2,
  36: 1.3,
  37: 1.4,
  38: 1.3,
  39: 1.5,
  4: 1.4,
  40

In [29]:
p ={}
#p['name'] = '홍'
p.update({'name':'홍'})
p

{'name': '홍'}

# 6절. 연습문제  : 고객관리
- 프로그램 시작시 'data/ch09_customers.txt (고객 데이터 백업용)' 의 내용을 load
                 파일이 없을 경우 빈 파일 생성
                 
- 'data/ch09_customers.txt (고객 데이터 백업용)'의 내용
    홍길도, 010-9999-9999, a@a.com, 30, 3, 까칠해
    홍길동, 010-9999-9999, a@a.com, 30, 3, 까칠해
    
## Customer 클래스 작성

In [169]:
class Customer:
    '고객데이터와 as_dict(), to_list_style (txt 백업시 필요), __str__()'
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc
        pass
        
    def as_dic(self): # 객체를 딕셔너리데이터로 반환 (csv 파일 저장시)
        return {
            'name':self.name,
            'phone':self.phone,
            'email':self.email,
            'age':self.age,
            'grade':self.grade,
            'etc':self.etc
            }
    
    def to_list_style(self): #객체를 list return('홍길동, 010-8999-9999, e@e.com, 20, 3, 까칠해'), format이나 join함수 사용
#         return '{}, {}, {}, {}, {}, {}'.format(
#             self.name,
#             self.phone,
#             self.email,
#             int(self.age),
#             int(self.grade),
#             self.etc)
        temp = [self.name,
            self.phone,
            self.email,
            str(self.age),
            str(self.grade),
            self.etc]
        return ', '.join(temp)
         
    def __str__(self): #  return '***홍길동 010-8999-9999 e@e.com 20 까칠해'
        return '{:>5}\t{:3}\t{:13}\t{:10}\t{:3}\t{}'.format('*'*self.grade,
                                          self.name,
                                          self.phone,
                                          self.email,
                                          self.age,
                                          self.etc)

def to_customer(row):
    '''
        txt파일 내용 한줄 row ='홍길동 , 010-8999-9999, e@e.com, 20, 3, 까칠해' (txt파일 내용)을 
        매개변수로 받아 Customer 객체로 반환
    '''
    data = row.split(',')

    name = data[0]
    phone = data[1].strip()
    email = data[2].strip()
    age = int(data[3].strip()) # strip() : 좌우 공백(space, \t, \n) 제거
    grade = int(data[4].strip())
    etc = data[5].strip()
    return Customer(name, phone, email, age, grade, etc)

In [132]:
c1 = Customer('홍길동', '010-8999-9999', 'e@e.com', 30, 3, '까칠해')
print(c1)
print('as_dic() =', c1.as_dic())
print('to_list_style() =', c1.to_list_style())
c2 = to_customer('홍길동, 010-8999-9999, e@e.com, 30, 3, 까칠해')
print(c2)

  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
as_dic() = {'name': '홍길동', 'phone': '010-8999-9999', 'email': 'e@e.com', 'age': 30, 'grade': 3, 'etc': '까칠해'}
to_list_style() = 홍길동, 010-8999-9999, e@e.com, 30, 3, 까칠해
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해


## 0. 처음 실행시 호출
- ch09_customer.txt (백업데이터)에서 customer_list return
- 파일이 없으면 빈 파일 생성, customer_list = [] return

In [133]:
# 0.실행하면 data/ch09_customers.txt 파일의 내용을 load(customer_list)
#      data/ch09_customers.txt이 존재하지 않으면
#          빈 data/ch09_customers.txt 파일을 생성하고
#              데이터는 customer_list=[]

def load_customers():
    customer_list = []
    try:
        with open('data/ch09_customers.txt', 'r', encoding='utf-8') as txt_f:
            # txt 데이터 한줄씩 customer 객체로 받아 customer.append
            lines = txt_f.readlines()
            for line in lines:
                # line = '홍길동, 010-8999-9999, e@e.com, 30, 3, 까칠해'
                customer = to_customer(line)
                # print(customer)
                customer_list.append(customer)

    except FileNotFoundError:
        with open('data/ch09_customers.txt', 'w', encoding='utf-8') as txt_f:
            print('초기화 파일을 생성했습니다')
    return customer_list

In [134]:
customer_list = load_customers()
print('데이터가 로드 되었습니다.')
for customer in customer_list:
    print(customer)

데이터가 로드 되었습니다.
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
*****	이길동	010-8999-9999	e@e.com   	 30	까칠해
   **	신길동	010-8999-9999	e@e.com   	 30	까칠해
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
*****	이길동	010-8999-9999	e@e.com   	 30	까칠해


## 1. 입력

In [168]:
# 1.입력
def fn1_insert_customer_info():
    '''
    사용자로부터 name, phone, email, age, grade, etc를 입력받아 Customer형 객체 변환
    '''
    import re
    name = input('이름 : ')
    name_pattern = r'[rk-glg]{2,}'
    while not re.search(name_pattern, name):
        print('이름을 제대로 입력하세요(한글 2글자 이상)')
        name = input('이름 : ')
    phone = input('전화번호 : ')
    email = input('이메일 : ')
    while True:
        try:
            age = int(input('나이 : '))
            if (age<0) | (age>130):
                raise Exception('나이 범위 이상')
            break
        except:
            print('올바른 나이를 입력하세요')

    try:
        grade = int(input('고객등급(1~5) : '))
        # grade = 1 if grade < 1 else 5 if grade> 5 else grade
        if grade < 1:
            grade = 1
        if grade > 5:
            grade = 5
    except:
        print('유효하지 않은 등급 입력시 1등급으로 초기화')
        grade = 1
        
    etc = input('기타정보 : ')

    return Customer(name, phone, email, age, grade, etc)

In [149]:
# 1번 함수 테스트
customer_list = load_customers()
customer = fn1_insert_customer_info()
customer_list.append(customer)
for customer in customer_list:
    print(customer)

이름 : ㅇㅇ
전화번호 : 1234
이메일 : ㄷ@ㅇㄹ
나이 : 34
고객등급(1~5) : 4
기타정보 : ㄹㄹ
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
*****	이길동	010-8999-9999	e@e.com   	 30	까칠해
   **	신길동	010-8999-9999	e@e.com   	 30	까칠해
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
*****	이길동	010-8999-9999	e@e.com   	 30	까칠해
 ****	ㅇㅇ 	1234         	ㄷ@ㅇㄹ      	 34	ㄹㄹ


## 2. 전체 출력

In [138]:
# 2.전체 출력
def fn2_print_customers(customer_list):
    'customer_list를 출력(pdf파일 40page 스타일)'
    print('='*70)
    print('{:^70}'.format('고객정보'))
    print('-'*70)
    print('{:>5}\t{:3}\t{:13}\t{:10}\t{:3}\t{}'.format('GRADE', '이름', '전화', '메일', '나이', '기타'))
    print('-'*70,) 
    
    for customer in customer_list:
        print(customer)
        
    print('='*70,'\n')

In [124]:
# 2번 기능 테스트
fn2_print_customers(customer_list)

                                 고객정보                                 
----------------------------------------------------------------------
GRADE	이름 	전화           	메일        	나이 	기타
----------------------------------------------------------------------
  ***	홍길동	010-8999-9999	e@e.com        	 30	까칠해
*****	이길동	010-8999-9999	e@e.com        	 30	까칠해
   **	신길동	010-8999-9999	e@e.com        	 30	까칠해
  ***	홍길동	010-8999-9999	e@e.com        	 30	까칠해
*****	이길동	010-8999-9999	e@e.com        	 30	까칠해
*****	ㅇㅇ 	1234         	kr@d.com       	 56	룰루



## 3. 삭제

In [145]:
# 3.삭제 (동명이인이 없을 경우)
def fn3_delete_customer(customer_list):
    '''
    삭제하고자 하는 고객이름을 input받아
    매개변수로 들어온 custoemr_list에서 삭제하고 "삭제했음/삭제 못했음"을 메세지로 출력
    '''
    del_name = input('삭제할 이름을 입력하세요 :')

    for customer in customer_list:
        if customer.name == del_name:
            customer_list.remove(customer)
            print('{}님 데이터를 삭제하였습니다'.format(del_name))
            return
    else:
        print('{}님 데이터는 존재하지 않습니다'.format(del_name))

In [148]:
# 3번 기능 테스트 (동명이인이 없을 경우)
customer_list = load_customers()
fn2_print_customers(customer_list)
fn3_delete_customer(customer_list)
fn2_print_customers(customer_list)

                                 고객정보                                 
----------------------------------------------------------------------
GRADE	이름 	전화           	메일        	나이 	기타
----------------------------------------------------------------------
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
*****	이길동	010-8999-9999	e@e.com   	 30	까칠해
   **	신길동	010-8999-9999	e@e.com   	 30	까칠해
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
*****	이길동	010-8999-9999	e@e.com   	 30	까칠해

삭제할 이름을 입력하세요 :이길동
이길동님 데이터를 삭제하였습니다
                                 고객정보                                 
----------------------------------------------------------------------
GRADE	이름 	전화           	메일        	나이 	기타
----------------------------------------------------------------------
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
   **	신길동	010-8999-9999	e@e.com   	 30	까칠해
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
*****	이길동	010-8999-9999	e@e.com   	 30	까칠해



In [158]:
# 3.삭제 (동명이인이 있을 경우)
def fn3_delete_customer(customer_list):
    '''
    삭제하고자 하는 고객이름을 input받아
    매개변수로 들어온 custoemr_list에서 삭제하고 "삭제했음/삭제 못했음"을 메세지로 출력
    '''
    del_name = input('삭제할 이름을 입력하세요 :')
    del_idx = [] # 삭제할 인덱스를 저장하는 용도
    for idx, customer in enumerate(customer_list):
        if customer.name == del_name:
            del_idx.append(idx)
            
    if del_idx:
        for idx in del_idx[::-1]:
            del customer_list[idx]
        print('{}님 데이터 {}개를 삭제하였습니다'.format(del_name, len(del_idx)))
    else:
        print('{}님 데이터는 존재하지 않습니다'.format(del_name))

In [160]:
# 3번 기능 테스트
customer_list = load_customers()
fn2_print_customers(customer_list)
fn3_delete_customer(customer_list)
fn2_print_customers(customer_list)

                                 고객정보                                 
----------------------------------------------------------------------
GRADE	이름 	전화           	메일        	나이 	기타
----------------------------------------------------------------------
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
*****	이길동	010-8999-9999	e@e.com   	 30	까칠해
   **	신길동	010-8999-9999	e@e.com   	 30	까칠해
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
*****	이길동	010-8999-9999	e@e.com   	 30	까칠해

삭제할 이름을 입력하세요 :신길동
신길동님 데이터 1개를 삭제하였습니다
                                 고객정보                                 
----------------------------------------------------------------------
GRADE	이름 	전화           	메일        	나이 	기타
----------------------------------------------------------------------
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
*****	이길동	010-8999-9999	e@e.com   	 30	까칠해
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
*****	이길동	010-8999-

## 4. 이름찾기

In [184]:
# 4.이름찾기 (동명이인이 있을 경우)
def fn4_search_customer(customer_list):
    '''
    찾고자 하는 이름을 input으로 받아, customer_list에서 검색하여
    같은 이름은 search_list에 append한 후 search_list를 출력, 
    같은 이름이 없으면 없다고 출력
    '''
    srch_name = input('출력할 이름을 입력하세요 :')
    srch_list = [] # 검색할 리스트를 저장하는 용도
    for customer in customer_list:
        if customer.name == srch_name:
            srch_list.append(customer)
            
    if srch_list:     
        print('{}님 데이터 {}개를 확인하였습니다'.format(srch_name, len(srch_list)))
        fn2_print_customers(srch_list)
    else:
        print('{}님 데이터는 존재하지 않습니다'.format(srch_name))

In [ ]:
# 4.이름찾기 (동명이인이 있을 경우, index를 사용하면?)
def fn4_search_customer(customer_list):
    '''
    찾고자 하는 이름을 input으로 받아, customer_list에서 검색하여
    같은 이름은 search_list에 append한 후 search_list를 출력, 
    같은 이름이 없으면 없다고 출력
    '''
    srch_name = input('출력할 이름을 입력하세요 :')
    srch_idx = [] # 검색할 리스트를 저장하는 용도
    for idx, customer in enumerate(customer_list):
        if customer.name == srch_name:
            srch_list.append(customer)
            
    if srch_list:     
        print('{}님 데이터 {}개를 확인하였습니다'.format(srch_name, len(srch_list)))
        fn2_print_customers(srch_list)
    else:
        print('{}님 데이터는 존재하지 않습니다'.format(srch_name))

In [183]:
# 4번 기능 테스트
customer_list = load_customers()
fn2_print_customers(customer_list)
fn4_search_customer(customer_list)
fn2_print_customers(customer_list)

                                 고객정보                                 
----------------------------------------------------------------------
GRADE	이름 	전화           	메일        	나이 	기타
----------------------------------------------------------------------
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
*****	이길동	010-8999-9999	e@e.com   	 30	까칠해
   **	신길동	010-8999-9999	e@e.com   	 30	까칠해
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
*****	이길동	010-8999-9999	e@e.com   	 30	까칠해

출력할 이름을 입력하세요 :ㅇㅇ
ㅇㅇ님 데이터는 존재하지 않습니다
                                 고객정보                                 
----------------------------------------------------------------------
GRADE	이름 	전화           	메일        	나이 	기타
----------------------------------------------------------------------
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
  ***	홍길동	010-8999-9999	e@e.com   	 30	까칠해
*****	이길동	010-8999-9999	e@e.com   	 30	까칠해
   **	신길동	010-8999-9999	e@e.com   	 30	까칠해
  ***	홍길동	010-8999-999

## 5. 내보내기(csv)

In [ ]:
# 5.내보내기 (CSV)
def fn5_save_customer_csv(customer_list):
    pass

In [89]:
# 5번 테스트

## 9. 종료

In [81]:
# 9.종료 (종료하기 전 customer_list 를 txt 파일에 저장하고 종료)
def fn9_save_customer_txt(customer_list):
    pass
    



In [ ]:
def main():
    global customer_list
    customer_list = load_customers() # ch09_customers.txt의 내용을 load
    while True:
        print("1:입력 "," 전체출력 "," 삭제 "," 이름찾기 "," 내보내기 (CSV)", " 종료 ", sep=' | ', end='')
        fn = int(input('메뉴선택 :'))
        if fn == 1:
            customer = fn1_insert_customer_info() # 입력받은 내용으로 customer 객체를 반환
            customer_list.append(customer)
        elif fn == 2:
            fn2_print_customers(customer_list)
        elif fn == 3:
            fn3_delete_customer(customer_list)
        elif fn == 4:
            fn4_search_customer(customer_list)
        elif fn == 5:
            fn5_save_customer_csv(customer_list)
        elif fn == 9:
            fn9_save_customer_txt(customer_list)
        break

if __name__ == '__main__':
    main()